In [1]:
%load_ext autoreload
%autoreload 2

import time
import cv2
import numpy as np
import tensorflow as tf
import imutils
import pandas as pd
from joblib import dump, load

from src.hands_finder import HandsFinder
from src.video_utils import VideoCapture, add_texts

objc[4834]: Class CaptureDelegate is implemented in both /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x169476480) and /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x295308860). One of the two will be used. Which one is undefined.
objc[4834]: Class CVWindow is implemented in both /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x1694764d0) and /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x2950dca68). One of the two will be used. Which one is undefined.
objc[4834]: Class CVView is implemented in both /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x1694764f8) and /opt/homebrew/Caskroom/miniforge/base/envs/env_tf/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.1

4.6.0


In [46]:
# Get this info from the trainer's notebook
classes = {
    "ok": 0,
    "victory": 1,
    "call": 2,
    "hang_in": 3,
    "one_up": 4,
    "two_up": 5,
    "hand_closed": 6,
    "hand_open": 7,
    "machedici": 8,
    "random": 9,
    "fuck you": 10,
}
classes = {v: k for k, v in classes.items()}
classes


{0: 'ok',
 1: 'victory',
 2: 'call',
 3: 'hang_in',
 4: 'one_up',
 5: 'two_up',
 6: 'hand_closed',
 7: 'hand_open',
 8: 'machedici',
 9: 'random',
 10: 'fuck you'}

## Prediction Logic

In [38]:
# Lightgbm
model_save_path = 'models/classifier.joblib'
clf = load(model_save_path) 

def prediction(arr):

  arr = arr.astype(np.float32)
  arr = arr.reshape(1, -1).round(4)
  predict_result = clf.predict_proba(arr)

  sq = np.squeeze(predict_result)
  idx = np.argmax(sq)
  pred = sq[idx]
  return idx, pred

In [47]:
# Tensorflow
tflite_save_path = 'models/lite.hdf5'

interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()
def prediction(arr):

  arr = arr.astype(np.float32)
  # Get I / O tensor
  input_details = interpreter.get_input_details()
  output_details = interpreter.get_output_details()

  interpreter.set_tensor(input_details[0]['index'], np.array([arr]))

  interpreter.invoke()
  tflite_results = interpreter.get_tensor(output_details[0]['index'])

  sq = np.squeeze(tflite_results)
  idx = np.argmax(sq)
  pred = sq[idx]
  return idx, pred

## Stuff to do

In [53]:
from src.teams_interaction import TeamsInteractions

teams = TeamsInteractions(use_keys_when_possible=False)

def do_your_stuff(classes, idx_pred):


    if classes[idx_pred] == "call":
        teams.meet_call_click()

    if classes[idx_pred] == "ok":
        teams.meet_send_reaction_click("ok")

    if classes[idx_pred] == "hand_open":
        teams.meet_send_reaction_click("raisehand")
        

## Main Loop

In [54]:

MIN_SCORE = 0.9

detector = HandsFinder()
vc = VideoCapture(fps=10)


CONTINUITY = 3
history_gestures = [None] * CONTINUITY
mod = -1

while vc.take_next():


    img = vc.get_frame()
   
    mod = 0 if (mod == CONTINUITY - 1) else mod + 1
    history_gestures[mod] = None # by default

    detector.fit(img)
    img = detector.transform_draw(img)
    img = detector.transform_connect_lines(img)
    img = detector.transform_square(img)

    if detector.found_hands():
      arr = detector.get_positions_normalized()[0]
      idx_pred, pred = prediction(arr)
      class_pred = classes[idx_pred]

      if pred >= MIN_SCORE and class_pred != "random":
        history_gestures[mod] = class_pred

        # Do I have the full history with the same gesture?
        if len(set(history_gestures)) == 1:
          add_texts(img, [str(classes[idx_pred]), str(pred)])

        do_your_stuff(classes, idx_pred)

    
    cv2.imshow("Image", img)

vc.finish()

Not found button on screen: meet_unmute.png
Not found button on screen: meet_unmute.png
Not found button on screen: meet_unmute.png
          Display Type: Built-In Retina LCD
          Resolution: 2560 x 1600 Retina
          Display Type: Built-In Retina LCD
          Resolution: 2560 x 1600 Retina
          Display Type: Built-In Retina LCD
          Resolution: 2560 x 1600 Retina
          Display Type: Built-In Retina LCD
          Resolution: 2560 x 1600 Retina
          Display Type: Built-In Retina LCD
          Resolution: 2560 x 1600 Retina
          Display Type: Built-In Retina LCD
          Resolution: 2560 x 1600 Retina
          Display Type: Built-In Retina LCD
          Resolution: 2560 x 1600 Retina
          Display Type: Built-In Retina LCD
          Resolution: 2560 x 1600 Retina
          Display Type: Built-In Retina LCD
          Resolution: 2560 x 1600 Retina
Not found button on screen: meet_open_reactions.png
Not found button on screen: meet_unmute.png
